In [ ]:
from surface_roughness import Surface
import numpy as np

surface = Surface(path='example_surface.stl')

In [ ]:
v0 = surface.points[surface.triangles[:,0]]
v1 = surface.points[surface.triangles[:,1]]
v2 = surface.points[surface.triangles[:,2]]

v1v0 = v1 - v0
v2v0 = v2 - v0

normals = np.cross(v1v0,v2v0,axisa=1,axisb=1)

areas = np.linalg.norm(normals,axis=1)
print(areas)
normals /= areas[:,np.newaxis]
print(normals)

In [ ]:
shear_dir = np.array([1,0,0])

proj = np.zeros_like(normals)
proj[:,0] = normals[:,0] - (shear_dir[1]**2*normals[:,0] - shear_dir[0]*shear_dir[1]*normals[:,1])
proj[:,1] = normals[:,1] - (shear_dir[0]*shear_dir[1]*normals[:,0] + shear_dir[0]**2 * normals[:,1])
proj[:,2] = normals[:,2]

proj /= np.linalg.norm(proj,axis=1)[:,np.newaxis]
app_dip = np.arccos(proj[:,0]*shear_dir[0]+proj[:,1]*shear_dir[1])-np.pi/2
print(app_dip)

In [ ]:
facing_areas = areas[app_dip>0]
facing_appdips = np.degrees(app_dip[app_dip>0])

delta_t = np.sum(facing_appdips * facing_areas) / facing_areas.sum()
delta_star_t = np.sqrt(np.sum(facing_areas * facing_appdips**2)/facing_areas.sum())
print(len(facing_areas))
print(delta_t)
print(delta_star_t)